In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from collections import defaultdict
from datetime import datetime
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [2]:
user_test_path = '../beauty_data/processed_data/user_test.json'
user_valid_path = '../beauty_data/processed_data/user_valid.json'
user_train_path = '../beauty_data/processed_data/user_train.json'

In [3]:
with open(user_train_path, 'rb') as f:
    user_train_dict = json.load(f)
print(len(user_train_dict))

624


In [4]:
with open(user_test_path, 'rb') as f:
    user_test_dict = json.load(f)
print(len(user_test_dict))

624


In [5]:
with open(user_valid_path, 'rb') as f:
    user_valid_dict = json.load(f)
print(len(user_valid_dict))

624


In [6]:
print(user_valid_dict['0'])
print(user_test_dict['0'])

[1, 2, 3, 4, 5, 5]
[2, 3, 4, 5, 5, 6]


In [7]:
item_information_path = '../beauty_data/processed_data/processed_item_information.json'
with open(item_information_path, 'r') as f:
    item_information_dict = json.load(f)
print(len(item_information_dict), item_information_dict['1'])

1216 {'title': 'Monoi - Monoi Pitate Jasmine 4 fl oz', 'brand': 'Monoi', 'price': '', 'description': '\n\nThe product is a jasmine-scented coconut oil, great for dry skin and hair, making them soft and non-oily. However, the jasmine scent may not be natural and not everyone may like it.'}


In [8]:
ratings_path = '../beauty_data/processed_data/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()

item  user  rating   timestamp
0     0     0     4.0  1155081600
1     1     0     5.0  1268697600
2     2     0     5.0  1404604800
3     3     0     5.0  1404604800
4     4     0     5.0  1404604800

In [9]:
user_profile_summary_path = '../beauty_data/processed_data/user_profile_summary.json'
with open(user_profile_summary_path, 'r') as f:
    user_profile_summary_dict = json.load(f)
print(len(user_profile_summary_dict), user_profile_summary_dict['1'])

160 

The user dislikes products that do not meet expectations, as seen with the 100% Cotton Premium Salon Towels. Despite being soft and thick, the towels unraveled after the first wash, which was disappointing for the user. The user seems to value durability and quality in their purchases.


### Creating zeroshot data

In [11]:
def get_item_description(item_information):
    item_details = ""
    if 'title' in item_information:
        item_details += item_information['title']
    # if 'brand' in item_information:
    #     item_details += f" Brand: {item_information['brand']}"
    # if 'price' in item_information:
    #     item_details += f" Price: {item_information['price']}"
    # if 'description' in item_information:
    #     item_details += f" Description: {item_information['description']}"
    return item_details

In [12]:
### Test Data
zeroshot_dict = dict()
check_missed_items = []
for user, item_list in user_test_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    prompt = """You are an expert beauty product recommender. You are provided with user's list of recent products that the user purchases and whether the user liked it or disliked it.
    User purchased the following items in the given order. List of recent items -
    """
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert beauty product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{target_item_details}" in atmost 100 words."""
    zeroshot_dict[user] = prompt
    # break

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [13]:
### Valid Data
zeroshot_valid_dict = dict()
check_missed_items = []
for user, item_list in user_valid_dict.items():
    # print(user, item_list)
    user_rating_dict = ratings_df[ratings_df['user'] == int(user)]
    # print(user_rating_dict)
    prompt = """You are an expert beauty product recommender. You are provided with user's list of recent products that the user purchases and whether the user liked it or disliked it.
    User purchased the following items in the given order. List of recent items -
    """
    for item in item_list[:-1]:
        try:
            user_item_rating = user_rating_dict[user_rating_dict['item'] == item]['rating'].values[0]
        except Exception as e:
            print(f"User: {user} Item: {item} error {e}")
            continue
        if user_item_rating > 3:
            ### Liked
            prompt += f"Liked {get_item_description(item_information_dict[str(item)])}\n"
        else:
            prompt += f"Disliked {get_item_description(item_information_dict[str(item)])}\n"
    target_item = item_list[-1]
    if str(target_item) not in item_information_dict:
        continue
    target_item_details = get_item_description(item_information_dict[str(target_item)])
    prompt += f"""As an expert beauty product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he liked and disliked about the products in at most 100 words.
    Explain with reasoning whether the user will like or dislike the next target item - "{target_item_details}" in atmost 100 words."""
    zeroshot_valid_dict[user] = prompt
    # break

User: 64 Item: 229 error index 0 is out of bounds for axis 0 with size 0
User: 215 Item: 399 error index 0 is out of bounds for axis 0 with size 0
User: 401 Item: 880 error index 0 is out of bounds for axis 0 with size 0
User: 427 Item: 399 error index 0 is out of bounds for axis 0 with size 0


In [14]:
print(len(zeroshot_dict), len(zeroshot_valid_dict))

622 624


In [15]:
print(zeroshot_dict['0'])
print("-"*100)
print(zeroshot_valid_dict['0'])

You are an expert beauty product recommender. You are provided with user's list of recent products that the user purchases and whether the user liked it or disliked it.
    User purchased the following items in the given order. List of recent items -
    Liked Yardley By Yardley Of London Unisexs Lay It On Thick Hand &amp; Foot Cream 5.3 Oz
Liked Fruits &amp; Passion Blue Refreshing Shower Gel - 6.7 fl. oz.
Liked Bonne Bell Smackers Bath and Body Starburst Collection
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
As an expert beauty product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by identifying the characteristics he like

In [16]:
total_len = 0
for user, prompt in zeroshot_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

220.17524115755626


In [17]:
users_very_long = []
for user, prompt in zeroshot_dict.items():
    if len(prompt.strip().split(' ')) > 400:
        users_very_long.append(user)
print(len(users_very_long))

1


In [18]:
for key in users_very_long:
    zeroshot_dict.pop(key, None)
print(len(zeroshot_dict))

621


In [19]:
total_len = 0
for user, prompt in zeroshot_valid_dict.items():
    total_len += len(prompt.strip().split(' '))
print(total_len/len(zeroshot_dict))

212.32367149758454


In [20]:
users_very_long = []
for user, prompt in zeroshot_valid_dict.items():
    if len(prompt.strip().split(' ')) > 400:
        users_very_long.append(user)
print(len(users_very_long))

0


In [21]:
for key in users_very_long:
    zeroshot_valid_dict.pop(key, None)
print(len(zeroshot_valid_dict))

624


In [22]:
zeroshot_valid_path = './ctr_zeroshot_dataset/zeroshot_valid.json'
with open(zeroshot_valid_path, 'w+') as f:
    json.dump(zeroshot_valid_dict, f)

In [23]:
zeroshot_test_path = './ctr_zeroshot_dataset/zeroshot_test.json'
with open(zeroshot_test_path, 'w+') as f:
    json.dump(zeroshot_dict, f)

In [24]:
ratings_df[ratings_df['user'] == int('0')]

item  user  rating   timestamp
0     0     0     4.0  1155081600
1     1     0     5.0  1268697600
2     2     0     5.0  1404604800
3     3     0     5.0  1404604800
4     4     0     5.0  1404604800
5     5     0     5.0  1404604800
6     5     0     5.0  1404604800
7     6     0     5.0  1461801600

In [25]:
test_prompt = zeroshot_dict['34']
print(test_prompt)
print("-"*100)
response = openai.completions.create(
    # model = "gpt-3.5-turbo-0125",
    model="gpt-3.5-turbo-instruct",
    prompt = test_prompt,
    temperature=0.3,
    max_tokens=256,
    # top_p=0.3,
    # frequency_penalty=0.5,
    # presence_penalty=0.5
)
print(response.choices[0].text)

You are an expert beauty product recommender. You are provided with user's list of recent products that the user purchases and whether the user liked it or disliked it.
    User purchased the following items in the given order. List of recent items -
    Liked Citre Shine Moisture Burst Shampoo - 16 fl oz
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
Liked Bath &amp; Body Works Ile De Tahiti Moana Coconut Vanille Moana Body Wash with Tamanoi 8.5 oz
As an expert beauty product recommender, do the following steps -
    Predict whether the user will like the target item or not in the following format - Prediction: Yes or No
    by analyzing the provided list of products purchased by the user in order and summarize the user behavior by iden